# Generate test data for package

## generate pgen files

In [ ]:
cd ~

In [ ]:
#R, 500 variants for each chromosome
for (i in 1:22){
  system(paste0("head -n 500 /home/simingz/causalTWAS/ukbiobank/ukb_chr", i, "_s40.22_snp.txt > test_chr", i, "_snp.txt"))

In [ ]:
# 2000 samples
head -n 2000 /home/simingz/causalTWAS/ukbiobank/ukbiobank_samples_s40.22.txt > test_samples.txt

In [ ]:
cmd = "cd /home/simingz/causalTWAS/ukbiobank/ukb_pgen_s40.22; plink2 --pfile ukb_chr{chrom}_s40.22_thinned --extract test_chr{chrom}_snp.txt --keep test_samples.txt --threads 5 --make-pgen --out test_chr{chrom}"
masterfile = "test_data-chr1to22.sh"
mf = open(masterfile, 'w')
for chrom in [str(i) for i in range(1,23)]:
  mf.write(cmd.format(chrom=chrom) + '\n')
mf.close()

In [ ]:
bash test_data-chr1to22.sh

In [1]:
#R, rename variant and sample
for (i in 1:22){
    df <- cbind(1:2000, 1:2000, sample(1:2, 2000, replace =T))
    colnames(df) <- c("#FID", "IID", "SEX")
    write.table(df , file= paste0("test_chr", i, ".psam") , row.names=F, col.names=T, sep="\t", quote = F)
}
for (i in 1:22){
    df <- data.table::fread(paste0("test_chr", i, ".pvar"), header =T)
    df$POS <- 1:nrow(df)
    df$ID <- paste0("rs", 1:nrow(df) + i * 1000)
    df[, c("REF", "ALT")] <- as.data.frame(t(sapply(1:nrow(df), function(x) c("A", "C", "T", "G")[sample(1:4,2, replace = T)])))
    write.table( df , file= paste0("test_chr", i, ".pvar") , row.names=F, col.names=T, sep="\t", quote = F)
}

## generate fusion weight files

In [ ]:
# R, wgt.RDat file and .pos file
# 20 genes, 1 gene in each of first 20 chromosomes
# first a few snps are eqtls, selected to have nonzero lasso weights.
wgtdir <- "/project2/mstephens/causalTWAS/fusion_weights/Adipose_Subcutaneous"
posfile <- paste0(wgtdir, ".pos")
posdf <- data.table::fread(posfile)[1:20]

for (i in 1:nrow(posdf)){
    load(file.path(dirname(wgtdir), posdf$WGT[i]))
    wgt.matrix <- wgt.matrix[abs(wgt.matrix[, "lasso"]) > 0, ]
    snps <- data.table::fread(paste0("/home/simingz/ctwas/inst/extdata/example_genotype_files/example_chr", i, ".pvar"))[1:nrow(wgt.matrix),]
    rownames(wgt.matrix) <- snps$ID
    snps$CM <- 0
    snps <- snps[, c("#CHROM", "ID", "CM", "POS","ALT" ,"REF")]
    colnames(snps) <- paste0("V", 1:6)
    snps <- data.frame(snps)
    N.tot <- 100
    save(wgt.matrix, snps, cv.performance, hsq, hsq.pv, N.tot, file = paste0("/home/simingz/ctwas/inst/extdata/example_fusion_weights/Tissue/Tissue.gene", i, ".wgt.RDat"))
}

posdf$PANEL <- "Tissue"
posdf$WGT <- paste0("Tissue/Tissue.gene", 1:20, ".wgt.RDat")
posdf$ID <- paste0("gene", 1:20)
posdf$CHR <- 1:20
posdf$P0 <- 1
posdf$P1 <- 30
posdf$N <- 100
write.table(posdf , file= "Tissue.pos" , row.names=F, col.names=T, sep="\t", quote = F)

## generate region files

In [ ]:
# 1 regions for each chr
regionlist <- list()
for (i in 1:22){
    regionlist[[i]] <- rbind(data.frame("chr" = i, start = 0, stop = 5),
                             data.frame("chr" = i, start = seq(5, 455, 30), stop = seq(35,505,30)))
                            
}
regions <- do.call(rbind, regionlist)
write.table(regions , file= "~/ctwas/inst/extdata/example_regions.bed"  , row.names=F, col.names=T, sep="\t", quote = F)